# Create dataset from yolo's dataset

In [ ]:
!unzip "/content/egg detection label.v1i.yolov8.zip" -d "/content/egg-detection-label-yolov8"

In [3]:
import os
import cv2

dataset_path = "/content/egg-detection-label-yolov8"
image_dir = "/content/egg-detection-label-yolov8/train/images"
labels_dir = "/content/egg-detection-label-yolov8/train/labels"
output_dir = "/content/egg-cropped"

os.makedirs(output_dir, exist_ok=True)

for label_file in os.listdir(labels_dir):
  if not label_file.endswith(".txt"):
    continue

  image_name = label_file.replace(".txt", ".jpg")
  image_path = os.path.join(image_dir, image_name)

  if not os.path.exists(image_path):
    continue

  img = cv2.imread(image_path)
  h, w, _ = img.shape

  with open(os.path.join(labels_dir, label_file), "r") as f:
    for i, line in enumerate(f):
      class_id, x_center, y_center, width, height = map(float, line.strip().split())

      x_center *= w
      y_center *= h
      width *= w
      height *= h

      x1 = int(x_center - width / 2)
      y1 = int(y_center - height / 2)
      x2 = int(x_center + width / 2)
      y2 = int(y_center + height / 2)

      crop = img[y1:y2, x1:x2]

      if crop.size == 0:
        continue

      # create folder
      class_folder = os.path.join(output_dir, f"class{int(class_id)}")
      os.makedirs(class_folder, exist_ok=True)

      # save img
      output_path = os.path.join(class_folder, f"{image_name[:-4]}_{i}.jpg")
      cv2.imwrite(output_path, crop)

In [4]:
!mv "/content/egg-cropped/class0" "/content/egg-cropped/Damaged"
!mv "/content/egg-cropped/class1" "/content/egg-cropped/Not Damaged"

In [5]:
dataset_dir = '/content/egg-cropped'

classes = os.listdir(dataset_dir)
print(f"classes: {classes}")

for cls in classes:
    cls_path = os.path.join(dataset_dir, cls)
    print(f"{cls}: {len(os.listdir(cls_path))} images")

classes: ['Damaged', 'Not Damaged']
Damaged: 728 images
Not Damaged: 177 images


# 2 Stages model

In [11]:
import shutil
from sklearn.model_selection import train_test_split

dataset_dir = "/content/egg-cropped"
output_dir = "/content/egg-split-cropped"

train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "val")
test_dir = os.path.join(output_dir, "test")

for d in [train_dir, val_dir, test_dir]:
  os.makedirs(d, exist_ok=True)

val_size = 0.15
test_size = 0.15

for class_name in os.listdir(dataset_dir):
  class_path = os.path.join(dataset_dir, class_name)
  if not os.path.isdir(class_path):
    continue

  images = os.listdir(class_path)
  images = [img for img in images if img.endswith(('.jpg'))]

  train_imgs, temp_imgs = train_test_split(images, test_size=(val_size + test_size), random_state=42)
  val_imgs, test_imgs = train_test_split(temp_imgs,test_size=test_size / (val_size + test_size), random_state=42)

  # copy files
  for split, split_imgs in zip(
      [train_dir, val_dir, test_dir],
      [train_imgs, val_imgs, test_imgs]
  ):
    split_class_dir = os.path.join(split, class_name)
    os.makedirs(split_class_dir, exist_ok=True)
    for img_name in split_imgs:
      shutil.copy(os.path.join(class_path, img_name), os.path.join(split_class_dir, img_name))



In [18]:
import tensorflow as tf

data_dir = '/content/egg-split-cropped'
IMAGE_SIZE = 224
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(data_dir, "train"),
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(data_dir, "val"),
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(data_dir, "test"),
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

# normalizing and caching
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(ds):
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
    return ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

train_ds = preprocess(train_ds)
val_ds = preprocess(val_ds)
test_ds = preprocess(test_ds)



Found 632 files belonging to 2 classes.
Found 136 files belonging to 2 classes.
Found 137 files belonging to 2 classes.


In [19]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomBrightness(factor=0.1),
])

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    data_augmentation,
    tf.keras.layers.Rescaling(1./255),

    tf.keras.layers.Conv2D(32, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_3 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,907,521 (91.20 MB)

 Trainable params: 23,907,521 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
  )

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 97ms/step - accuracy: 0.8065 - loss: 0.5890 - val_accuracy: 0.8015 - val_loss: 0.4996
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.8100 - loss: 0.5063 - val_accuracy: 0.8015 - val_loss: 0.5605
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.8189 - loss: 0.5296 - val_accuracy: 0.8015 - val_loss: 0.5042
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.8307 - loss: 0.4834 - val_accuracy: 0.8015 - val_loss: 0.5025
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.7805 - loss: 0.5605 - val_accuracy: 0.8015 - val_loss: 0.5058
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.8068 - loss: 0.5052 - val_accuracy: 0.8015 - val_loss: 0.5019
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.7842 - loss: 0.5324 - val_accuracy: 0.8015 - val_loss: 0.5056
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.8250 - loss: 0.4937 - val_accuracy: 0.8015 - v